In [1]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
import pprint
import highway_env  # noqa: F401

TRAIN = True

n_cpu = 6
batch_size = 64
# env = make_vec_env("racetrack-v0", n_envs=n_cpu, vec_env_cls=SubprocVecEnv)

In [2]:
from racetrack_config import *


print(env.spec)
pprint.pprint(env.config)

EnvSpec(id='racetrack-v0', entry_point='highway_env.envs:RacetrackEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=None, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={'render_mode': 'rgb_array'}, namespace=None, name='racetrack', version=0, additional_wrappers=(), vector_entry_point=None)
{'action': {'lateral': True, 'longitudinal': False, 'type': 'ContinuousAction'},
 'action_reward': -0.3,
 'centering_position': [0.5, 0.5],
 'collision_reward': -1,
 'controlled_vehicles': 1,
 'duration': 300,
 'lane_centering_cost': 4,
 'lane_centering_reward': 1,
 'manual_control': False,
 'observation': {'align_to_vehicle_axes': True,
                 'as_image': False,
                 'features': ['presence', 'on_road'],
                 'grid_size': [[-18, 18], [-18, 18]],
                 'grid_step': [3, 3],
                 'type': 'OccupancyGrid'},
 'offscreen_rendering': False,
 'other_vehicles': 1,
 'other_vehic

/Users/BOULLIERERWAN1/opt/anaconda3/envs/rl_py311/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.config to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.config` for environment variables or `env.get_wrapper_attr('config')` that will search the reminding wrappers.
  logger.warn(


## Train model and save it

In [4]:
model = PPO(
    "MlpPolicy",
    env,
    policy_kwargs=dict(net_arch=[dict(pi=[256, 256], vf=[256, 256])]),
    n_steps=batch_size * 12 // n_cpu,
    batch_size=batch_size,
    n_epochs=10,
    learning_rate=5e-4,
    gamma=0.9,
    verbose=2,
    tensorboard_log="racetrack_ppo/",
)
# Train the model
if TRAIN:
    model.learn(total_timesteps=int(1e2)) # try with 1e5 but takes a very long time!!
    model.save("racetrack_ppo/model")
    # del model

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to racetrack_ppo/PPO_6


/Users/BOULLIERERWAN1/opt/anaconda3/envs/rl_py311/lib/python3.11/site-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


----------------------------
| time/              |     |
|    fps             | 20  |
|    iterations      | 1   |
|    time_elapsed    | 6   |
|    total_timesteps | 128 |
----------------------------


## Vizualise in a video

In [5]:
# Run the algorithm
model = PPO.load("racetrack_ppo/model", env=env)

env = gym.make("racetrack-v0")
env = RecordVideo(
    env, video_folder="racetrack_ppo/videos", episode_trigger=lambda e: True
)
env.unwrapped.set_record_video_wrapper(env)

for video in range(100):
    done = truncated = False
    obs, info = env.reset()
    while not (done or truncated):
        # Predict
        action, _states = model.predict(obs, deterministic=True)
        # Get reward
        obs, reward, done, truncated, info = env.step(action)
        # Render
        env.render()
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


ValueError: Render mode is None, which is incompatible with RecordVideo. Initialize your environment with a render_mode that returns an image, such as rgb_array.